# Week 3: Machine learning
Voor deze week opdracht is er gekozen voor de dataset [Gender Recognition by Voice](https://www.kaggle.com/datasets/primaryobjects/voicegender). 

Als template voor de opdracht heb ik gebruik gemaakt van het example bestand knn. Dit gaf al een redelijke basis om de opdracht uit te voeren.

In [1]:
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Als variabeles voor de subset heb ik gekozen voor: 
- label: male or female
- meanfreq: mean frequency (in kHz)
- meanfun: average of fundamental frequency measured across acoustic signal
- meandom: average of dominant frequency measured across acoustic signal
- dfrange: range of dominant frequency measured across acoustic signal
- modindx: modulation index. Calculated as the accumulated absolute difference between adjacent measurements of fundamental frequencies divided by the frequency range
- IQR: interquantile range (in kHz)
- kurt: kurtosis (see note in specprop description)

In [4]:
df = pd.read_csv('voice.csv')
df = df.dropna()

#maken van subset voor het uitvoeren van de opdracht
df2 = df[["label","meanfreq", "meanfun", "meandom", "dfrange", "modindx", "IQR", "kurt" ]] 
df2.head()

,label,meanfreq,meanfun,meandom,dfrange,modindx,IQR,kurt
0,male,0.059781,0.084279,0.007812,0.000000,0.000000,0.075122,274.402906
1,male,0.066009,0.107937,0.009014,0.046875,0.052632,0.073252,634.613855
2,male,0.077316,0.098706,0.007990,0.007812,0.046512,0.123207,1024.927705
3,male,0.151228,0.088965,0.201497,0.554688,0.247119,0.111374,4.177296
4,male,0.135120,0.106398,0.712812,5.476562,0.208274,0.127325,4.333713


In [3]:
#laat de hoeveelheid van beide genders zien
df['label'].value_counts() 

male      1584
female    1584
Name: label, dtype: int64

Hieronder begin ik met set uppen van machine learning proces. Ik pak alle variabelen behalve 'label' als x waarde. Label wordt gebruikt als y waarde, omdat deze variabele aangeeft welke gender het is. Omdat de variabele niet allemaal op dezelfde manier is geschreven voeren we ook nog een normalisatie uit om het trainen makkelijker te maken. Vervolgens splitsen we de dataset in 4 verschillende variabele om het systeem te trainen. 

In [5]:
from sklearn.preprocessing import normalize 

#aanmaken van de x matrix 
X = df2[["meanfreq", "meanfun", "meandom", "dfrange", "modindx", "IQR", "kurt" ]] 

#Normaliseren van de x matrix, zorgt ervoor dat alles op dezelfde schaal is.
X = normalize(X) 

#aanmaken van y variabelen 
y = df2['label'] 

#splitsen van data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 

In [6]:
from sklearn.neighbors import KNeighborsClassifier 

#aanmaken van knn classifier met een waarde van 5 neighbors
knn = KNeighborsClassifier(n_neighbors=5) 
knn = knn.fit(X_train, y_train) 

#uitrekenen hoe goed de test data past. 
knn.score(X_test, y_test)

0.7129337539432177

71% van genders wordt juist gepredict aan de hand van de gekozen variabelen.

In [8]:
from sklearn.metrics import confusion_matrix
y_test_pred = knn.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_test_pred) #creates a "confusion matrix"
cm

array([[326, 131],
       [142, 352]], dtype=int64)

In [9]:
#In order to read it easily , let's make a dataframe out of it, and add labels to it.
conf_matrix = pd.DataFrame(cm, index=["Male","Female"], columns = ["Male_p","Female_p"]) 
conf_matrix

,Male_p,Female_p
Male,326,131
Female,142,352


De _recall_ en _precision_ voor de identificatie van mannen en vrouwen zijn als volgt:

$recall (man) = \frac{326}{326 + 131} = .71$

$recall (vrouw) = \frac{352}{142 + 352} = .71$

$precision (man) = \frac{326}{326 + 142} = .70$

$precision (vrouw) = \frac{352}{352 + 131} = .73$

